# Inspect Data Processing Logic

### Import

In [1]:
import sys
sys.path.append('source')
from data_processor import DataProcessor

### Initialise

In [2]:
processor = DataProcessor(
    'question_bank.md',
    'data/trees_df.csv',
    'data/patches_df.csv'
)

In [3]:
processor.load_data()

### Parse Questions

In [4]:
intents = processor.parse_question_bank()
intents

{'level_requirement': ['(What / Which) is the best tree (to grow / to plant / to choose) at level ___?',
  '(What / Which) are the best trees (to grow / to plant / to choose) at level ___?',
  '(What / Which) (should I / could I / do I / can I) (grow / plant / choose) at level ___?',
  '(What / Which) tree is the best (to grow / to plant / to choose) at level ___?',
  '(What / Which) trees are best (to grow / to plant / to choose) at level ___?',
  '(What / Which) trees are the best (to grow / to plant / to choose) at level ___?',
  '(What / Which) tree(s) (should I / could I / do I / can I) (grow / plant / choose) at level ___?',
  '(What / Which) level (do I need / is needed / is required) to (grow / plant) ___?',
  '(What / Which) level (do I need / is needed / is required) for (growing / planting) ___?',
  'What is the level (requirement / required / needed) to (grow / plant) ___?',
  'What is the level (requirement / required / needed) for (growing / planting) ___?',
  'What is (r

In [5]:
intents_list = list(intents)
intents_list

['level_requirement', 'payment', 'growth_time', 'patches', 'transportation']

In [6]:
intent = intents_list[0]
intent

'level_requirement'

In [7]:
template = intents[intent][0]
template

'(What / Which) is the best tree (to grow / to plant / to choose) at level ___?'

### Expand Template

In [8]:
expanded_list = processor.expand_template(template)
expanded_list

['What is the best tree to grow at level ___?',
 'What is the best tree to plant at level ___?',
 'What is the best tree to choose at level ___?',
 'Which is the best tree to grow at level ___?',
 'Which is the best tree to plant at level ___?',
 'Which is the best tree to choose at level ___?']

In [9]:
expanded = expanded_list[0]
expanded

'What is the best tree to grow at level ___?'

### Fill Template

In [10]:
filled = processor.fill_template(expanded, intent)
filled

['What is the best tree to grow at level 15?',
 'What is the best tree to grow at level 30?',
 'What is the best tree to grow at level 45?',
 'What is the best tree to grow at level 60?',
 'What is the best tree to grow at level 75?',
 'What is the best tree to grow at level 27?',
 'What is the best tree to grow at level 33?',
 'What is the best tree to grow at level 39?',
 'What is the best tree to grow at level 42?',
 'What is the best tree to grow at level 51?',
 'What is the best tree to grow at level 57?',
 'What is the best tree to grow at level 68?',
 'What is the best tree to grow at level 81?']

# Preview Training Data

In [11]:
texts, labels = processor.generate_training_data(20)
print('\n'.join(texts))

What level do I need to grow palm?
What is the level needed to plant apple?
What level do I need to grow oak?
What tree is the best to grow at level 15?
What level are banana?
What should I choose at level 51?
Which trees should I plant at level 57?
Which trees do I plant at level 81?
What trees can I choose at level 33?
What trees could I plant at level 30?
What is required to grow yew?
What do I need to grow yew?
Which trees could I grow at level 51?
What level are curry?
What can I plant at level 60?
What is the best tree to grow at level 57?
What trees are the best to grow at level 57?
What should I plant at level 15?
Which trees should I grow at level 75?
Which are the best trees to choose at level 39?
Which item does the farmer need for growing dragonfruit?
What payment do I give to the farmer to grow willow?
Which gift should I give to grow orange?
What gift does the farmer require for growing pineapple?
What gift do farmers need to plant curry?
What item does the farmer require